## Carbon dioxide

In order to compare our measurements to modelled data for carbon dioxide ($CO_2$), we should use the high time resolution workflow. Thia allows us to extract and use the correct footprints and emissions to account for the natural diurnal cycle.

In [1]:
import os
import matplotlib.pyplot as plt
from openghg.localclient import get_obs_surface
from openghg.modules import Emissions, FOOTPRINTS, ObsSurface
from openghg.processing import footprints_data_merge, search

# object_store = "/work/<username>/object_store"

# Or to create a temporary object store if testing on your local machine
import tempfile
tmp_dir = tempfile.TemporaryDirectory()
object_store = tmp_dir.name  # "/tmp/openghg_store"

os.environ["OPENGHG_PATH"] = object_store

## Loading the data files

In [2]:
## Defining inputs

site = "TAC"
network = "DECC"
height = "185m"

domain = "EUROPE"
species = "co2"

In [3]:
## Observations

obs_file = "/home/rt17603/shared/obs_raw/AGAGE_GCWerks/data-picarro/tacolneston/tac.picarro.1minute.185m.dat"
ObsSurface.read_file(filepath=obs_file, data_type="CRDS", site=site, network=network, inlet=height)

Processing: tac.picarro.1minute.185m.dat: 100%|██████████| 1/1 [00:20<00:00, 20.95s/it]


defaultdict(dict,
            {'processed': {'tac.picarro.1minute.185m.dat': {'ch4': 'b6c3abfa-970b-4e9a-b377-f8c48c1b0100',
               'co2': 'fe3517dc-beca-48e1-af2c-888248ac7fca'}}})

In [4]:
## Footprints

# #fp_file_path = "/home/rt17603/shared/LPDM/fp_NAME/EUROPE/TAC-185magl_UKV_co2_EUROPE_201401.nc" - file doesn't exist at the moment
fp_file_path = "/work/rt17603/NAME_files/co2_test/TAC-185magl_UKV_co2_EUROPE_201407.nc"

model = "NAME"
metmodel = "UKV"

FOOTPRINTS.read_file(filepath=fp_file_path, site=site, height=height, domain=domain, model=model, metmodel=metmodel, species=species)

{'tac_europe_NAME_185m': '74bbfab3-4e70-42b3-887e-181d4e290c1a'}

In [5]:
## Emissions

flux_file_path = "/home/rt17603/shared/LPDM/emissions/EUROPE/co2-rtot-cardamom-2hr_EUROPE_2014.nc"

source = "rtot-cardamom"
date = "2014"

Emissions.read_file(filepath=flux_file_path, species=species, source=source, domain=domain, date=date, high_time_resolution=True)

{'co2_rtotcardamom_europe_2014': '1bf58eb0-fa98-4330-b5e2-458997a92928'}

## Combining data

In [6]:
start_date = "2014-07-01"
end_date = "2014-08-01"

CombinedData_HiTRes = footprints_data_merge(site=site, height=height, domain=domain, network=network,
                                            start_date=start_date, end_date=end_date, flux_sources=source,
                                            species=species, load_flux=True, calc_timeseries=True,
                                            time_resolution="high")


/work/rt17603/environments/openghg/lib/python3.7/site-packages/pandas/core/indexes/base.py:5277: FutureWarning: Indexing a timezone-naive DatetimeIndex with a timezone-aware datetime is deprecated and will raise KeyError in a future version.  Use a timezone-naive object instead.
  start_slice, end_slice = self.slice_locs(start, end, step=step, kind=kind)
  1%|          | 4/743 [00:00<00:21, 34.57it/s]

Calculating modelled timeseries comparison:


100%|██████████| 743/743 [00:21<00:00, 34.16it/s]


In [9]:
CombinedData_HiTRes.data

<xarray.Dataset>
Dimensions:                    (H_back: 14, height: 20, lat: 293, lev: 1, lon: 391, time: 743)
Coordinates:
  * time                       (time) datetime64[ns] 2014-07-01T00:00:30 ... ...
  * lat                        (lat) float32 10.73 10.96 11.2 ... 78.82 79.06
  * lon                        (lon) float32 -97.9 -97.55 -97.2 ... 39.03 39.38
  * H_back                     (H_back) int64 0 2 4 6 8 10 ... 16 18 20 22 24 26
  * lev                        (lev) |S1 b'F'
  * height                     (height) float32 500.0 1.5e+03 ... 1.95e+04
Data variables: (12/22)
    mf                         (time) float64 394.9 394.8 394.9 ... 387.6 390.2
    mf_variability             (time) float64 0.0455 0.01933 ... 0.02244 0.5133
    mf_number_of_observations  (time) float64 19.17 19.11 19.17 ... 19.11 19.11
    fp_HiTRes                  (lat, lon, H_back, time) float64 nan nan ... 0.0
    fp                         (lat, lon, time) float32 nan 0.0 0.0 ... 0.0 0.0
    temperature                (time) float32 nan 13.04 12.4 ... 18.11 19.11
    ...                         ...
    mean_age_particles_n       (height, lon, time) float32 nan 0.0 ... 0.0 0.0
    mean_age_particles_e       (height, lat, time) float32 nan 0.0 ... 0.0 0.0
    mean_age_particles_s       (height, lon, time) float32 nan 0.0 ... 0.0 0.0
    mean_age_particles_w       (height, lat, time) float32 nan 0.0 ... 0.0 0.0
    mf_mod                     (time) float64 0.0 5.925 7.25 ... 11.75 12.89
    mf_mod_high_res            (time) float64 nan nan ... nan 1.173e-05